<div align = "center">

# **Exploración de datos (Base siniestros)**

</div>

## Librerias

In [26]:
library(readxl)
library(dplyr)

## Data

In [27]:
Siniestros <- read_excel("../data/input/Siniestros_Hist.xlsx")

In [28]:
# Ver nombres actuales del dataframe Siniestros
names(Siniestros)

[1] "FECHASIN"         "VLRPRIMAPAG"      "VLRPRISUSCR"      "VLRASEGU"        
 [5] "VLRASEGU_ORIG"    "VLRPAGADO."       "VLRSININCUR"      "VLRDEDUCIBLE"    
 [9] "VLRRECOBRO"       "VLRRSVACONSTIAMP" "VLRRSVAPAGAMP"    "FECPAGOAMP"      
[13] "ESTADO_FINAL"     "COBERTURA_FINAL"

In [ ]:
# Método directo para renombrar columnas
names(Siniestros) <- c(
  "Fecha_siniestro",
  "Prima_efectivamente_pagada_hasta_fecha_siniestro",
  "Prima_suscrita_incluida_cobertura_responsabilidad_civil",
  "Asegurado_otras_coberturas_distintas_responsabilidad_civil",
  "Asegurado_original",
  "Efectivamente_pagado_por_siniestro",
  "Siniestro_incurrido",
  "Deducible",
  "Recobro_obtenido",
  "Reserva_constituida_para_amparo",
  "Reserva_pagada_para_amparo",
  "Fecha_pagbho_amparo",
  "Estado_final_siniestro",
  "Cobertura_final_aplicada"
)

In [30]:
unique(Siniestros$Cobertura_final_aplicada)

[1] "PTH"       "PPD"       "RC BIENES" "PPH"       "RC PERS"

In [31]:
# Cambiar los niveles de la columna Cobertura_final_aplicada
Siniestros$Cobertura_final_aplicada <- factor(Siniestros$Cobertura_final_aplicada, 
                                           levels = c("PTH", "PPD", "RC BIENES", "PPH", "RC PERS"),
                                           labels = c("Perdida_total_hurto", 
                                                     "Perdida_parcial_danos",
                                                     "Responsabilidad_civil_bienes",
                                                     "Perdida_parcial_hurto",
                                                     "Responsabilidad_civil_personas"))

In [35]:
# Verificar los cambios
levels(Siniestros$Cobertura_final_aplicada)

[1] "Perdida_total_hurto"            "Perdida_parcial_danos"         
[3] "Responsabilidad_civil_bienes"   "Perdida_parcial_hurto"         
[5] "Responsabilidad_civil_personas"

In [33]:
head(Siniestros)

Fecha_siniestro,Prima_efectivamente_pagada_hasta_fecha_siniestro,Prima_suscrita_incluida_cobertura_responsabilidad_civil,Asegurado_otras_coberturas_distintas_responsabilidad_civil,Asegurado_original,Efectivamente_pagado_por_siniestro,Siniestro_incurrido,Deducible,Recobro_obtenido,Reserva_constituida_para_amparo,Reserva_pagada_para_amparo,Fecha_pago_amparo,Estado_final_siniestro,Cobertura_final_aplicada
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
2017-01-01,300000,300000,4.350e+06,4.350e+06,0,3262500,870000,0,4.350e+06,3480000,20171220,Pagado,Perdida_total_hurto
2017-01-01,5543164,16639492,8.400e+07,8.400e+07,4601532,4601532,737717,0,8.400e+07,0,20170215,Pagado,Perdida_parcial_danos
2017-01-01,1072019,1072019,5.255e+07,5.255e+07,3375620,3375620,737717,375909,5.255e+07,0,20170131,Pagado,Perdida_parcial_danos
2017-01-01,958821,958821,3.449e+07,3.449e+07,3600357,3600357,737717,0,3.449e+07,0,20170131,Pagado,Perdida_parcial_danos
2017-01-01,714176,3398721,2.000e+09,2.000e+09,0,2175223,0,1241379,2.000e+09,2175223,NA,Avisado,Perdida_parcial_danos
2017-01-01,1681845,5193649,1.500e+09,1.500e+09,1569899,1569899,737717,0,1.500e+09,0,20170213,Pagado,Perdida_parcial_danos


In [34]:
# Primero corregir el problema de variables - usar Siniestros consistentemente
# Crear columnas de fecha
Siniestros$Año <- format(Siniestros$Fecha_siniestro, "%Y")
Siniestros$Mes <- format(Siniestros$Fecha_siniestro, "%m")
Siniestros$Día <- format(Siniestros$Fecha_siniestro, "%d")

# Verificar los tipos de cobertura únicos
print("Tipos de cobertura únicos:")
print(unique(Siniestros$Cobertura_final_aplicada))

[1] "Tipos de cobertura únicos:"
[1] Perdida_total_hurto            Perdida_parcial_danos         
[3] Responsabilidad_civil_bienes   Perdida_parcial_hurto         
[5] Responsabilidad_civil_personas
5 Levels: Perdida_total_hurto ... Responsabilidad_civil_personas


In [36]:
# Tabla 1: Cantidad de siniestros por fecha y tipo de cobertura
library(tidyr)

tabla_cantidad <- Siniestros %>%
  filter(!is.na(Cobertura_final_aplicada)) %>%  # Excluir NA
  group_by(Año, Mes, Día, Cobertura_final_aplicada) %>%
  summarise(Cantidad_siniestros = n(), .groups = 'drop') %>%
  pivot_wider(names_from = Cobertura_final_aplicada, 
              values_from = Cantidad_siniestros, 
              values_fill = 0)

print("Tabla 1: Cantidad de siniestros por fecha y tipo de cobertura")
head(tabla_cantidad)

[1] "Tabla 1: Cantidad de siniestros por fecha y tipo de cobertura"


Año,Mes,Día,Perdida_total_hurto,Perdida_parcial_danos,Responsabilidad_civil_bienes,Perdida_parcial_hurto,Responsabilidad_civil_personas
<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
2017,01,01,1,52,2,0,0
2017,01,02,0,59,4,1,0
2017,01,03,2,55,9,7,0
2017,01,04,0,33,6,6,0
2017,01,05,1,51,12,4,0
2017,01,06,2,48,8,2,0


In [37]:
# Tabla 2: Valor efectivamente pagado por fecha y tipo de cobertura
tabla_valor <- Siniestros %>%
  filter(!is.na(Cobertura_final_aplicada)) %>%  # Excluir NA
  group_by(Año, Mes, Día, Cobertura_final_aplicada) %>%
  summarise(Valor_pagado = sum(Efectivamente_pagado_por_siniestro, na.rm = TRUE), .groups = 'drop') %>%
  pivot_wider(names_from = Cobertura_final_aplicada, 
              values_from = Valor_pagado, 
              values_fill = 0)

print("Tabla 2: Valor efectivamente pagado por fecha y tipo de cobertura")
head(tabla_valor)

[1] "Tabla 2: Valor efectivamente pagado por fecha y tipo de cobertura"


Año,Mes,Día,Perdida_total_hurto,Perdida_parcial_danos,Responsabilidad_civil_bienes,Perdida_parcial_hurto,Responsabilidad_civil_personas
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2017,01,01,0,133517846,797543,0,0
2017,01,02,0,169847277,3368686,5264237,0
2017,01,03,3562500,171924150,8986920,6710874,0
2017,01,04,0,72116936,5534291,8304611,0
2017,01,05,0,163020188,18795563,8977343,0
2017,01,06,3292500,129012714,11878226,1866013,0


In [38]:
# Verificar las dimensiones de las tablas
print(paste("Dimensiones tabla cantidad:", dim(tabla_cantidad)[1], "x", dim(tabla_cantidad)[2]))
print(paste("Dimensiones tabla valor:", dim(tabla_valor)[1], "x", dim(tabla_valor)[2]))

# Mostrar resumen de las tablas
print("\nResumen tabla cantidad:")
summary(tabla_cantidad)

print("\nResumen tabla valor:")
summary(tabla_valor)

[1] "Dimensiones tabla cantidad: 396 x 8"
[1] "Dimensiones tabla valor: 396 x 8"
[1] "\nResumen tabla cantidad:"


     Año                Mes                Día            Perdida_total_hurto
 Length:396         Length:396         Length:396         Min.   :0.00       
 Class :character   Class :character   Class :character   1st Qu.:1.00       
 Mode  :character   Mode  :character   Mode  :character   Median :1.00       
                                                          Mean   :1.71       
                                                          3rd Qu.:3.00       
                                                          Max.   :7.00       
 Perdida_parcial_danos Responsabilidad_civil_bienes Perdida_parcial_hurto
 Min.   :24.00         Min.   : 1.000               Min.   :0.000        
 1st Qu.:40.00         1st Qu.: 6.000               1st Qu.:1.000        
 Median :45.00         Median : 8.000               Median :2.000        
 Mean   :45.52         Mean   : 8.553               Mean   :2.101        
 3rd Qu.:51.00         3rd Qu.:11.000               3rd Qu.:3.000        
 Max.   :7

[1] "\nResumen tabla valor:"


     Año                Mes                Día            Perdida_total_hurto
 Length:396         Length:396         Length:396         Min.   :        0  
 Class :character   Class :character   Class :character   1st Qu.:        0  
 Mode  :character   Mode  :character   Mode  :character   Median :  9896861  
                                                          Mean   : 31005042  
                                                          3rd Qu.: 47476943  
                                                          Max.   :315926592  
 Perdida_parcial_danos Responsabilidad_civil_bienes Perdida_parcial_hurto
 Min.   : 72116936     Min.   :       0             Min.   :       0     
 1st Qu.:124432916     1st Qu.: 6781886             1st Qu.:  243151     
 Median :153200075     Median :10929142             Median : 2216796     
 Mean   :158797212     Mean   :12442331             Mean   : 3522130     
 3rd Qu.:185072905     3rd Qu.:16439194             3rd Qu.: 5161655     
 Max.   :4

In [39]:
# Guardar las tablas pivoteadas como archivos CSV
write.csv(tabla_cantidad, "../data/processed/tabla_cantidad.csv", row.names = FALSE)
write.csv(tabla_valor, "../data/processed/tabla_valor.csv", row.names = FALSE)

print("Tablas guardadas exitosamente:")
print("- tabla_cantidad.csv en data/processed/")
print("- tabla_valor.csv en data/processed/")

[1] "Tablas guardadas exitosamente:"
[1] "- tabla_cantidad.csv en data/processed/"
[1] "- tabla_valor.csv en data/processed/"
